# Token Count Statistics

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display
import __paths__
import notebooks.pos_statistics.tokens_count_gui as tokens_count_gui

gui = tokens_count_gui.create_token_count_gui("SSI")
display(gui.layout())